<a href="https://colab.research.google.com/github/ChiragBellara/Video-Translation-Using-Deep-Learning/blob/silbunsa-patch-1/TTS_with_bark/TTS_bark_TextArrayConv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Compared to the "TTS_bark_main" this module takes a json file and gets an array of texts and then convert them to the desired language audio file, it could be used if we want to manipulate our video according to timestamps. So the idea would be to take each subtitle from a video, convert them to audio and then append them to the same timestamps on the video.
To run the code, an user will need to provide the path to the json file which should contain
[
  {
      "start_time": 0.0,
      "end_time": 10.0,
      "text": " In the last 50 years, we have made progress. No one can refuse this."
  },
  {
      "start_time": 10.0,
      "end_time": 15.0,
      "text": " During the elections, we have been demanding votes,"
  }
]
format and then run the "convert" function with desired sample rate and language

In [1]:
!pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-0w7aesfz
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-0w7aesfz
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.0 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=2567413

In [2]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-z9eqc9uz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-z9eqc9uz
  Resolved https://github.com/huggingface/transformers.git to commit 2c658b5a4282f2e824b4e23dc3bcda7ef27d5827
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8119970 sha256=948f75841a722726e159685e2947f21a3c7de85724bbc6a256ab5795979bd5a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-lr6i7_dg/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [3]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
import scipy
# download and load all models
preload_models()


text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 104MB/s]


In [7]:
##function to get the text array from the json file

import json
speech_array=[]

def get_speech(file_path):
  with open(file_path) as user_file:
    file_contents = user_file.read()
  parsed_json = json.loads(file_contents)
  for entry in parsed_json:
    speech_array.append(entry["text"])

In [8]:
f="speech.json"
get_speech(f)

In [10]:
import os
import soundfile as sf

def convert(speech_array_path, lang, sample_rt):
  n=1
  duration=[]
  file_path=[]
  speaker = {
    "en": "v2/en_speaker_3",
    "hi": "v2/hi_speaker_2",
    "ja": "v2/ja_speaker_6",
    "fr": "v2/fr_speaker_6",
    "es": "v2/es_speaker_6"
    }

  SPEAKER = speaker[lang]
  for speech in speech_array:

    audio_array = generate_audio(speech, history_prompt=SPEAKER)
    sf.write(f"output_audio{n}.wav", audio_array, samplerate=sample_rt)
    file_path.append(os.path.abspath(f"output_audio{n}.wav"))
    n=n+1

  return file_path


In [11]:
f_paths=convert(speech_array,"en",28000)

100%|██████████| 11/11 [00:10<00:00,  1.08it/s]
